In [1]:
import pickle
import bs4
import gzip
import tarfile

from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer, HashingVectorizer, TfidfVectorizer

from bs4 import BeautifulSoup
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
from pprint import pprint

In [31]:
#
# Read all tar-files in folder (parent)
#
def get_tars():
    parent = "/home/hades/exjobb/dumps/dnm/"
    files = [parent+f for f in listdir(parent) if isfile(join(parent, f))]
    return files

In [32]:
#
# Untar a file and return its members
# returns: tar, members
#
def open_tar(fn):
    tar = tarfile.open(fn, "r:xz")
    members = tar.getmembers()
    return tar, members

In [41]:
#
# Open untared member
#
def open_mem(tar, mem):
#     tar = tarfile.open(tar, "r:xz")
    fn = tar.extractfile(mem)
    f = fn.read()
    f = f.decode(errors='ignore')
    raw = clean_html(f)
    return raw

In [54]:
#
# 1 - Get a list of tar-files (ret: tars)
# 2 - Get members from arg
# 3 - Get cleaned html-text from arg
#
tars = get_tars()
print("Tar-files:",len(tars))

tar, members = open_tar(tars[86]) #tars[35] = alphabay
print("Tar-file members:",len(members))

raw = open_mem(tar, members[0])
print("Untared-file clean text len:",len(raw))

Tar-files: 163
Tar-file members: 932994
Untared-file clean text len: 6171


In [52]:
#
# Look for a substr in tars
#
for i,x in enumerate(tars):
    if "silkroad2" in x:
        print(i,x)

21 /home/hades/exjobb/dumps/dnm/silkroad2-20140129-sohhlz-vendors.tar.xz
30 /home/hades/exjobb/dumps/dnm/silkroad2-20140927-daryllau.tar.xz
32 /home/hades/exjobb/dumps/dnm/silkroad2-forums-2014093020141016-rasmusandersen.tar.xz
55 /home/hades/exjobb/dumps/dnm/silkroad2.tar.xz
86 /home/hades/exjobb/dumps/dnm/silkroad2-forums.tar.xz
101 /home/hades/exjobb/dumps/dnm/silkroad2-forums-20140419-whom-astorposts.tar.xz


In [ ]:
# hasher = HashingVectorizer()
count_vect = CountVectorizer()

In [ ]:
#
# Store content from tar in list
# 
content = []
for fn in tqdm(members):
    raw = open_mem(tar, fn)
    content.append(raw)

In [ ]:
feat1 = count_vect.fit_transform([raw])
feat2 = count_vect.fit_transform([raw2])

print(feat1.shape)
print(feat2.shape)

In [28]:
pprint(hasher.vocabulary_)

{'01': 0,
 '05': 1,
 '06': 2,
 '075': 3,
 '08': 4,
 '10': 5,
 '14': 6,
 '2013': 7,
 '2014': 8,
 '21': 9,
 '24': 10,
 '26': 11,
 '29': 12,
 '48': 13,
 'account': 14,
 'actions': 15,
 'active': 16,
 'add': 17,
 'age': 18,
 'am': 19,
 'april': 20,
 'august': 21,
 'buddy': 22,
 'date': 23,
 'day': 24,
 'for': 25,
 'forum': 26,
 'forums': 27,
 'gwern': 28,
 'hello': 29,
 'help': 30,
 'home': 31,
 'info': 32,
 'january': 33,
 'karma': 34,
 'last': 35,
 'list': 36,
 'local': 37,
 'logout': 38,
 'machines': 39,
 'memberlist': 40,
 'members': 41,
 'message': 42,
 'messages': 43,
 'my': 44,
 'new': 45,
 'newnewbie': 46,
 'news': 47,
 'of': 48,
 'offline': 49,
 'openmarket': 50,
 'pandora': 51,
 'per': 52,
 'personal': 53,
 'pm': 54,
 'posts': 55,
 'profile': 56,
 'read': 57,
 'registered': 58,
 'replies': 59,
 'rss': 60,
 'search': 61,
 'send': 62,
 'settings': 63,
 'show': 64,
 'simple': 65,
 'since': 66,
 'smf': 67,
 'stats': 68,
 'stckit00': 69,
 'summary': 70,
 'the': 71,
 'time': 72,
 'to':

In [ ]:
test1 = TfidfTransformer(use_idf=0).fit_transform(feat1)
test2 = TfidfTransformer(use_idf=0).fit_transform(feat2)
test3 = TfidfTransformer(use_idf=0).fit_transform(feat1+feat2)


print(test1.sum())
print(test2.sum())

print(test3.sum())
print(test1.sum()+test2.sum())

In [ ]:
#
# Read 
#
mypath = "C:/Users/Hampus/Downloads/Torrents/dnmarchives/alphabay/alphabay/2015-05-06/"
onlyfiles = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]

print("Files found:",len(onlyfiles))
print(onlyfiles[0])

In [ ]:
print("Files found:",len(onlyfiles))
new_f = []
for x in onlyfiles:
    if "listing" in x:
         new_f.append(x.split("&")[0])
    
onlyfiles = list(set(new_f))
print("Unique files found:",len(onlyfiles))

In [68]:
raw_content = []

for f in tqdm(onlyfiles):
    try:
        with open(f, encoding="ascii", errors='replace') as f:
            raw = f.read()
            raw_content.append(raw)
    except:
        0


100%|██████████████████████████████████████| 5879/5879 [03:16<00:00, 29.88it/s]


In [10]:
#
# Removes script and html-tags - returns plain text
#
def clean_html(str):
    soup = BeautifulSoup(str,"lxml")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract() # rip it out 
    
    text = soup.get_text(strip=False)
    
    text=text.replace(u'\xa0',u'')

    return text

In [70]:
#
# Get substring start index
#
def substr(str, subs, start=0):
    for i in range(start,len(str)+start):
        if str[i:i+len(subs)] == subs:
            return i
    raise ValueError('No substring:',subs)
    
def rev_substr(str, subs, start):
    for i in range(start,0,-1):
        if str[i:i+len(subs)] == subs:
            return i
    raise ValueError('No rev substring:',subs)
  

In [71]:
#
# Retireve country origin text
# text string
#
def get_country(str):
    ind_start = substr(str, "FeaturesOrigin country", 150)+22
    ind_stop = substr(str, "Ship", ind_start) #?
    
    txt = str[ind_start:ind_stop]
    return txt 

In [72]:
#
# Retireve country origin text
# text string
#
def get_shipsto(str):
    ind_start = substr(str, "Ships to", 150)+8
    ind_stop = substr(str, "\n", ind_start)
    
    txt = str[ind_start:ind_stop]
    return txt 

In [73]:
#
# Retireve price
# text string
#
def get_price(str):
    ind_start = substr(str, "Purchase price: ", 150)+8
    ind_stop = substr(str, "Qty:", ind_start)
    
    txt = str[ind_start:ind_stop]
    return txt 

In [74]:
#
# Retireve country origin text
# text string
#
def get_soldby(str):
    ind_start = substr(str, "Sold by", 150)+7
    ind_stop = substr(str, "-", ind_start)
    
    txt = str[ind_start:ind_stop]
    return txt 


In [75]:
#
# Retireve Alphabay listing title. 
# title string
#
def get_title(str):
    ind = substr(str, " | AlphaBay Market")
    return str[:ind]

In [76]:
#
# Retireve Alphabay listing category. 
# title string
#
def get_category(str):
    ind_start = substr(str, "Home /")+6
    mid_stop = substr(str, "Listing Options", ind_start)
    ind_stop = rev_substr(str, "\n", mid_stop)
    cate = str[ind_start:ind_stop]
    cate = cate.replace("\n", "")
    cate = cate.replace(" ", "")
    return cate 
    

In [77]:
#
# Retireve description text
# text string
#
def get_desc(str):
    ind_start = substr(str, "Product Description", 150)+19
    ind_stop = substr(str, "Listing Bids", ind_start)#-21
    
    txt = str[ind_start:ind_stop]

    return txt 
    

In [78]:
dataset = []
err=0

for page in tqdm(raw_content):
    tmp = {}
    txt = clean_html(page)
    try:
        title = get_title(txt)
        cate = get_category(txt)
        desc = get_desc(txt)
        country = get_country(txt)
        soldby = get_soldby(txt)
        price = get_price(txt)

        tmp["title"] = title
        tmp["category"] = cate
        tmp["soldby"] = soldby
        tmp["price"] = price
        tmp["country"] = country
        tmp["description"] = desc

        dataset.append(tmp)
    except:
        err+=1
print(err)


100%|██████████████████████████████████████| 5876/5876 [05:30<00:00, 17.78it/s]


293


In [4]:
pickle.dump(dataset, open("dataset_alphabay.p", "wb"), protocol=2)

NameError: name 'dataset' is not defined

In [80]:
print("files:",len(raw_content))
print("Listings:",(len(raw_content)-err),"(",(len(raw_content)-err)/len(raw_content)*100,"%)")
print("Other:",err,"(",err/len(raw_content)*100,"%)")



files: 5876
Listings: 5583 ( 95.0136147038802 %)
Other: 293 ( 4.986385296119809 %)


In [83]:
txt = clean_html(raw_content[124])

title = get_title(txt)
cate = get_category(txt)
desc = get_desc(txt)
origin = get_country(txt)
soldby = get_soldby(txt)
price = get_price(txt)

print("title:",title)
print("_____________")
print("category:",cate)
print("_____________")
print("sold by:",soldby)
print("_____________")
print("price:",price)
print("_____________")
print("country:",origin)

print("_____________")
print("description:",desc)

title: ������Special request  bank drop account ������
_____________
category: Fraud/Accounts&BankDrops/
_____________
sold by:  docmomo99 
_____________
price:  price: USD 130.00
_____________
country:  Worldwide
_____________
description: ���������������The best service, the cheapest and smartest choice���������������

This account aren't hacked or phished. They are created by me, which make you the only owner of the account.
----------------------------------------------------------------------------------------------------------
A provide info about the person: SSN+DOB+MMN  DL exp and issue and more.


-- PACKAGES --

[���] CHECKING ACCOUNT WORKING 
[���] PAYMENT PROCESSORS READY
[���] GOOGLE VOICE NUMBER
[���] IP
[���] FULLZ DETAILS
[���] DEBIT CARD SHIP TO YOUR DROP
[���] LOGIN PASSWORD ANS SECRETS QUESTIONS. 
[���] COINBASE + MINT LINK  OR NOT ITS YOUR CHOICE.

[���] FREQUENTLY ASKED QUESTIONS

Q1. Do you have any in stock or do I need to wait?
A1. I Have only one for the moment

In [ ]:
# import tarfile
# tar = tarfile.open('C:/Users/Hampus/Downloads/Torrents/dnmarchives/tom.tar.xz', 'r:xz')
# gz = f.read()
# f.close()

In [ ]:
# with tarfile.open('C:/Users/Hampus/Downloads/Torrents/dnmarchives/tom.tar.xz') as f:
#     f.extractall('.')